# Setup and Imports

In [1]:
# !pip install xgboost
# !pip install lightgbm
# !pip install catboost

In [2]:
import sys
if '../Toolkit' not in sys.path: sys.path.append('../Toolkit')

%load_ext autoreload
%autoreload 1

%aimport tools
%aimport models

import tools as t
import models as m

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
from IPython.display import clear_output

import numpy as np
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 999
pd.options.display.max_rows = 113
import matplotlib.pyplot as plt
import seaborn as sns

RS = 35577 # global random state seed
raw_data_path = 'data-raw'
processed_data_path = 'data-processed'

In [3]:
import time

from sklearn.preprocessing import StandardScaler, PowerTransformer

In [4]:
(X, y) = t.from_pkl(f'{processed_data_path}/all_b1_b2.data.pkl')

X.shape
X.columns

y.shape
y.value_counts()

(200, 36)

Index(['x__amin', 'x__amax', 'x__sum', 'x__median', 'x__mean', 'x__std',
       'x__var', 'x__q1', 'x__q3', 'x__iqr', 'x__kurtosis', 'x__skew',
       'y__amin', 'y__amax', 'y__sum', 'y__median', 'y__mean', 'y__std',
       'y__var', 'y__q1', 'y__q3', 'y__iqr', 'y__kurtosis', 'y__skew',
       'z__amin', 'z__amax', 'z__sum', 'z__median', 'z__mean', 'z__std',
       'z__var', 'z__q1', 'z__q3', 'z__iqr', 'z__kurtosis', 'z__skew'],
      dtype='object')

(200,)

0    100
1    100
Name: status, dtype: int64

# CV

## Multiple Models

https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

# from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score, log_loss

from sklearn.model_selection import KFold, RepeatedKFold, StratifiedKFold, RepeatedStratifiedKFold

# First Run

In [6]:
# all_results = []
# n_est_list = [200]
# models_and_params = [
#     (DecisionTreeClassifier, {}),
#     (KNeighborsClassifier, {}),
    
#     *[ (RandomForestClassifier, {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
#     *[ (AdaBoostClassifier,     {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
    
#     *[ (LGBMClassifier,         {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
#     *[ (XGBClassifier,          {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
#     *[ (CatBoostClassifier,     {'random_state': RS, 'n_estimators': n_e, 'silent': True}) for n_e in n_est_list ],
# ]
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)
# metrics = [ roc_auc_score, log_loss ]

# r = t.grid_exec(
#     lambda model: m.cv_classification(model, cv, X, y, metrics), 
#     models_and_params, 
#     on1Completed = lambda r: m.display_stats(r)
# )


In [7]:
all_results = []
n_est_list = [2000]
models_and_params = [
    # (DecisionTreeClassifier, {}),
    # (KNeighborsClassifier, {}),
    
    # *[ (RandomForestClassifier, {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
    *[ (AdaBoostClassifier,     {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
    
    *[ (LGBMClassifier,         {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
    # *[ (XGBClassifier,          {'random_state': RS, 'n_estimators': n_e}) for n_e in n_est_list ],
    # *[ (CatBoostClassifier,     {'random_state': RS, 'n_estimators': n_e, 'silent': True}) for n_e in n_est_list ],
]
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2)
metrics = [ roc_auc_score, log_loss ]

r = t.grid_exec(
    lambda model: m.cv_classification(model, cv, X, y, metrics), 
    models_and_params, 
    on1Completed = lambda r: m.display_stats(r)
)


,model,params,n_folds,roc_auc_mean,roc_auc_std,log_loss_mean,log_loss_std,time
0,AdaBoostClassifier,'n_estimators': 2000,10,0.992,0.010,0.452,0.025,32.4
1,LGBMClassifier,'n_estimators': 2000,10,0.989,0.010,0.228,0.160,15.1


In [8]:
# all_f_imp = []

# n_folds = 5
# n_models = len(all_trained_models[::5])


# for i, m in enumerate(models_to_analyse):
#     if 'feature_names_in_' in m.__dict__.keys():
#         all_f_imp.append(
#             pd.Series(data = m.feature_importances_, index = m.feature_names_in_, name=f'{m.__class__.__name__[0]}_{i}')#.rank().astype(int)
#         )
#     else:
#         all_f_imp.append(
#             pd.Series(data = m.feature_importances_, index = m.feature_name_, name=f'{m.__class__.__name__[0]}_{i}')
#         )

# df_f_imp = pd.concat(all_f_imp, axis=1)
# df_f_imp


In [9]:
# fe = pd.concat(all_f_imp, axis=1)
# sfor s_col in range(1,3):
#     fe.insert(5, 'sum_1', None)
#     fe.insert(11, 'sum_2', None)
#     fe.insert(17, 'sum_3', None)
#     fe['sum_1'] = fe.iloc[:, :5].sum(axis=1)
#     fe['sum_2'] = fe.iloc[:, 5:10].sum(axis=1)
#     fe['sum_3'] = fe.iloc[:, 10:].sum(axis=1)


# fe.sort_values(by=['sum_1', 'sum_2', 'sum_3'], ascending=False)\
#     .round(2)\
#     .style\
#     .format('{:.2f}')\
#     .highlight_max(color='green')\
#     .highlight_min(color='blue')\
#     .highlight_max(color='red', subset=['sum_1', 'sum_2', 'sum_3'])

In [10]:
# fe_s = fe.sort_values(by=['sum_1', 'sum_2', 'sum_3'], ascending=False).round(2).style
# _ = fe_s.format('{:.2f}')
# for c in fe.columns:
#     _ = fe_s.background_gradient(cmap='Greens', subset=c, vmin=-5, gmap=fe[c].rank(), )
    
# for c in ['sum_1', 'sum_2', 'sum_3']:
#     _ = fe_s.background_gradient(cmap='Reds', subset=c, vmin=-50, gmap=fe[c].rank(), )
    
# fe_s
#     # .highlight_max(color='green')\
#     # .highlight_min(color='blue')\

#     # .highlight_max(color='red', subset=['sum_1', 'sum_2', 'sum_3'])